In [3]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [4]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['image_20.jpg',
 'image_40.jpg',
 'models',
 'anh.png',
 'testanh.png',
 '27.ipynb',
 'FaceImage',
 'data_face_features.pickle',
 '31c_data_preprocessing.ipynb']

In [5]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [6]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.0076054   0.09255902  0.06909025 ...  0.13474342  0.13193654
    0.0550187 ]]

 [[ 0.0076054   0.09255902  0.06909025 ...  0.13474342  0.13193654
    0.0550187 ]]

 [[ 0.07423197  0.19272773  0.05085204 ... -0.06173938  0.1379407
    0.00699797]]

 ...

 [[ 0.0430265   0.09132998 -0.14244998 ...  0.08621342  0.07641552
    0.03136611]]

 [[ 0.03505763  0.17826444 -0.08103755 ...  0.00479291  0.07760584
    0.08672322]]

 [[ 0.01884078  0.21863489 -0.08203609 ... -0.07726788  0.05244384
    0.04946936]]]
['1721031615_VuDinhNguyen' '1721031615_VuDinhNguyen'
 '1721031615_VuDinhNguyen' ... '162000339_HuynhAnhDuy'
 '162000339_HuynhAnhDuy' '162000339_HuynhAnhDuy']


In [7]:
x.shape

(4695, 1, 128)

In [8]:
y.shape

(4695,)

In [9]:
x = x.reshape(-1, 128)

In [10]:
x.shape

(4695, 128)

In [11]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4225, 128), (470, 128), (4225,), (470,))

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [14]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [15]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [16]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.73
Accurency Test = 0.72
F1 Score Train = 0.52
F1 Score Test = 0.51


In [17]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [18]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.92
Accurency Test = 0.86
F1 Score Train = 0.85
F1 Score Test = 0.76


In [19]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [20]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.66


In [21]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [22]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [23]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.86
F1 Score Train = 0.97
F1 Score Test = 0.79


In [24]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [25]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   9.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   9.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   8.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.8s
[

KeyboardInterrupt: 

In [ ]:
model_best_estimator = model_grid.best_estimator_

In [ ]:
model_grid.best_score_

In [ ]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))